In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from model_ALT import MyCatBoostModel

In [ ]:
cols = ['diagnosis', 'bpm', 'sdnn', 'rmssd', 'hr_mad', 'hf', 'rsa']

my_dataset = pd.DataFrame(columns=cols)
my_disorders = set()

data_path = os.getcwd() + '/data_ALT'

for data_file in os.listdir(data_path):
    dataset = pd.read_csv(data_path + '/' + data_file)

    dataset = dataset.dropna()
    dataset = dataset.reindex(columns=cols)

    diagnosis = data_file.split('__')[0]
    my_disorders.add(diagnosis)
    dataset['diagnosis'] = diagnosis

    my_dataset = pd.concat(
        [my_dataset, dataset],
    ignore_index=True)

print(my_disorders)

{'CONTROL', 'BIPOLAR', 'SCHIZO', 'DEPRESSION'}


/tmp/ipykernel_631673/3694097567.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  my_dataset = pd.concat(


In [3]:
train, test = train_test_split(
    my_dataset.to_numpy(),
    test_size=0.2,
    stratify=my_dataset[['diagnosis']].to_numpy(),
    shuffle=True,
    random_state=190525
)
train, val = train_test_split(
    train,
    test_size=0.1,
    stratify=train[:, 0],
    shuffle=True,
    random_state=190525
)

train_X = train[:, 1:]
train_y = train[:, 0]
val_X = val[:, 1:]
val_y = val[:, 0]
test_X = test[:, 1:]
test_y = test[:, 0]

print(train_X.shape)
print(train_y.shape)
print(np.unique(train_y))

(162, 6)
(162,)
['BIPOLAR' 'CONTROL' 'DEPRESSION' 'SCHIZO']


In [4]:
m = MyCatBoostModel(2000, list(my_disorders))
m.forward(train_X, train_y, val_X, val_y)

Начинаю 1vA-тренировку для CONTROL...
0:	learn: 0.6399217	test: 0.6430983	best: 0.6430983 (0)	total: 92.1ms	remaining: 3m 4s
100:	learn: 0.0240822	test: 0.1946562	best: 0.1796456 (67)	total: 5.73s	remaining: 1m 47s
200:	learn: 0.0097568	test: 0.2292670	best: 0.1796456 (67)	total: 13.1s	remaining: 1m 56s
300:	learn: 0.0058616	test: 0.2538745	best: 0.1796456 (67)	total: 21s	remaining: 1m 58s
400:	learn: 0.0042716	test: 0.2694959	best: 0.1796456 (67)	total: 29.3s	remaining: 1m 56s
500:	learn: 0.0034172	test: 0.2815836	best: 0.1796456 (67)	total: 37.3s	remaining: 1m 51s
bestTest = 0.1796455516
bestIteration = 67
Shrink model to first 68 iterations.
Тренировка весов для CONTROL завершена!

Начинаю 1vA-тренировку для BIPOLAR...
0:	learn: 0.6564961	test: 0.6598405	best: 0.6598405 (0)	total: 71.4ms	remaining: 2m 22s
100:	learn: 0.1501597	test: 0.3202584	best: 0.3162170 (89)	total: 7.3s	remaining: 2m 17s
200:	learn: 0.0798711	test: 0.3296942	best: 0.3162170 (89)	total: 15.2s	remaining: 2m 16s
3

In [5]:
importances = m.get_importances()

for i in range(len(my_disorders)):
    print(f'=== {list(my_disorders)[i]} ===')
    for j in range(len(importances[i])):
        print(f'{cols[j+1]} -- {importances[i][j]}')

=== CONTROL ===
bpm -- 7.391761241653948
sdnn -- 8.003032308301362
rmssd -- 3.8495148259904406
hr_mad -- 77.12499099792296
hf -- 2.326219509654539
rsa -- 1.304481116476733
=== BIPOLAR ===
bpm -- 13.527586103523326
sdnn -- 27.657339767737266
rmssd -- 13.963220415418176
hr_mad -- 24.373204120135895
hf -- 9.568722890395362
rsa -- 10.909926702789956
=== SCHIZO ===
bpm -- 16.211258567702057
sdnn -- 12.566827800635192
rmssd -- 21.7459941203426
hr_mad -- 22.85964240542689
hf -- 15.530747328594487
rsa -- 11.085529777298772
=== DEPRESSION ===
bpm -- 39.23121517528068
sdnn -- 21.487919707011386
rmssd -- 15.019654364720417
hr_mad -- 10.12291718630771
hf -- 9.810855448594772
rsa -- 4.327438118085046


In [6]:
m.set_mode('test')

results = m.forward(test_X, test_y)
results

/home/wsl/ИИ ПРОЕКТ/ХреНоваяПапка/model.py:118: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)


(['CONTROL accuracy is 0.9782608695652174',
  'BIPOLAR accuracy is 0.782608695652174',
  'SCHIZO accuracy is 0.782608695652174',
  'DEPRESSION accuracy is 0.9782608695652174'],
 ['CONTROL f1-score is 0.972972972972973',
  'BIPOLAR f1-score is 0.375',
  'SCHIZO f1-score is 0.7222222222222222',
  'DEPRESSION f1-score is nan'],
 ['CONTROL precision is 0.9473684210526315',
  'BIPOLAR precision is 0.42857142857142855',
  'SCHIZO precision is 0.7222222222222222',
  'DEPRESSION precision is nan'],
 ['CONTROL recall is 1.0',
  'BIPOLAR recall is 0.3333333333333333',
  'SCHIZO recall is 0.7222222222222222',
  'DEPRESSION recall is 0.0'],
 ['CONTROL specificity is 0.9642857142857143',
  'BIPOLAR specificity is 0.8918918918918919',
  'SCHIZO specificity is 0.8214285714285714',
  'DEPRESSION specificity is 1.0'])

In [7]:
m.save_weights("w")

Веса для CONTROL сохранены в "w__CONTROL.cbm"
Веса для BIPOLAR сохранены в "w__BIPOLAR.cbm"
Веса для SCHIZO сохранены в "w__SCHIZO.cbm"
Веса для DEPRESSION сохранены в "w__DEPRESSION.cbm"
